DA 626 QUESTION-8 210104108

In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install transformers datasets torch scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset

Load Data

In [4]:


# Load train data
train_data = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt', sep=':::', engine='python', names=["ID", "Title", "Genre", "Description"])

# Load test data (without Genre)
test_data = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt', sep=':::', engine='python', names=["ID", "Title", "Description"])

# Combine Title and Description for training and test data
train_data['Text'] = train_data['Title'] + " " + train_data['Description']
test_data['Text'] = test_data['Title'] + " " + test_data['Description']

In [5]:
print(train_data)

          ID                                         Title          Genre  \
0          1                 Oscar et la dame rose (2009)          drama    
1          2                                 Cupid (1997)       thriller    
2          3             Young, Wild and Wonderful (1980)          adult    
3          4                        The Secret Sin (1915)          drama    
4          5                       The Unrecovered (2007)          drama    
...      ...                                           ...            ...   
54209  54210                              "Bonino" (1953)         comedy    
54210  54211                  Dead Girls Don't Cry (????)         horror    
54211  54212    Ronald Goedemondt: Ze bestaan echt (2008)    documentary    
54212  54213                     Make Your Own Bed (1944)         comedy    
54213  54214   Nature's Fury: Storm of the Century (2006)        history    

                                             Description  \
0       Listeni

In [6]:
print(test_data)

          ID                             Title  \
0          1             Edgar's Lunch (1998)    
1          2         La guerra de papá (1977)    
2          3      Off the Beaten Track (2010)    
3          4           Meu Amigo Hindu (2015)    
4          5                Er nu zhai (1955)    
...      ...                               ...   
54195  54196   "Tales of Light & Dark" (2013)    
54196  54197      Der letzte Mohikaner (1965)    
54197  54198              Oliver Twink (2007)    
54198  54199                Slipstream (1973)    
54199  54200        Curitiba Zero Grau (2010)    

                                             Description  \
0       L.R. Brane loves his life - his car, his apar...   
1       Spain, March 1964: Quico is a very naughty ch...   
2       One year in the life of Albin and his family ...   
3       His father has died, he hasn't spoken with hi...   
4       Before he was known internationally as a mart...   
...                                    

In [7]:
# Step 2: Encode the labels in the training data
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['Genre'])

In [8]:
# Step 3: Tokenize the data using BERT's tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    return tokenizer(data['Text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize_data(train_data)
test_encodings = tokenize_data(test_data)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Step 4: Create a PyTorch dataset class
class MovieDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = MovieDataset(train_encodings, train_labels)
test_dataset = MovieDataset(test_encodings)

In [10]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Initialize the BERT model for classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_strategy="no",
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Fine-tune the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.140600,No log
2,0.802500,No log
3,0.452300,No log


TrainOutput(global_step=5085, training_loss=0.8490875491702099, metrics={'train_runtime': 4334.4358, 'train_samples_per_second': 37.523, 'train_steps_per_second': 1.173, 'total_flos': 1.070062844306688e+16, 'train_loss': 0.8490875491702099, 'epoch': 3.0})

In [12]:
train_preds = trainer.predict(train_dataset)
train_pred_labels = train_preds.predictions.argmax(axis=1)

# Calculate accuracy on the training dataset

In [13]:
#  from sklearn.metrics import accuracy_score
#     val_predictions = trainer.predict(val_dataset)
#     preds = np.argmax(val_predictions.predictions, axis=1)

#     # Calculate accuracy
#     accuracy = accuracy_score(y_val, preds)
#     fold_accuracies.append(accuracy)
#     print(f'Fold Accuracy: {accuracy:.4f}')

# # Calculate and print average accuracy across all folds
# average_accuracy = np.mean(fold_accuracies)
# print(f'Average K-Fold Accuracy: {average_accuracy:.4f}')

In [14]:
from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(train_labels, train_pred_labels)
print(f"Training Accuracy: {train_accuracy:.4f}")

Training Accuracy: 0.9250


In [15]:
# Get predictions
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)

# Convert label indices back to genre names
predicted_genres = label_encoder.inverse_transform(predicted_labels)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [16]:
print(label_encoder.classes_)  # To see all the genres that were encoded

[' action ' ' adult ' ' adventure ' ' animation ' ' biography ' ' comedy '
 ' crime ' ' documentary ' ' drama ' ' family ' ' fantasy ' ' game-show '
 ' history ' ' horror ' ' music ' ' musical ' ' mystery ' ' news '
 ' reality-tv ' ' romance ' ' sci-fi ' ' short ' ' sport ' ' talk-show '
 ' thriller ' ' war ' ' western ']


In [17]:
#to avoid space trailing issue
predicted_genres_cleaned = [genre.strip() for genre in predicted_genres]

# Create the submission DataFrame with cleaned genres
submission_df = pd.DataFrame({
    'ID': test_data['ID'],
    'genre': predicted_genres_cleaned
})

# Check the unique values in the cleaned predictions
print(set(predicted_genres_cleaned))  # Display unique cleaned genres

# Verify the submission DataFrame
print(submission_df.head(10))

submission_df.to_csv('submission_final.csv', index=False)

{'adult', 'romance', 'talk-show', 'family', 'game-show', 'documentary', 'biography', 'thriller', 'musical', 'horror', 'sci-fi', 'western', 'fantasy', 'action', 'short', 'history', 'sport', 'animation', 'mystery', 'war', 'news', 'adventure', 'drama', 'comedy', 'crime', 'music', 'reality-tv'}
   ID        genre
0   1     thriller
1   2       comedy
2   3  documentary
3   4        drama
4   5        drama
5   6     thriller
6   7        drama
7   8       comedy
8   9  documentary
9  10        drama
